# 상장기업정보 수집하기

In [2]:
import pandas as pd
import time
import requests
import re
from bs4 import BeautifulSoup as bs

In [3]:
def info_extraction(trs):
    page_result = [] 
    for tr in trs: 
        stock_type = tr.select_one("td:nth-child(1) > img")['alt']
        company_name = tr.select_one("td:nth-child(1) a")['title']
        code = tr.select_one("td:nth-child(1) a")['onclick'].split("'")[1]
        code = f"{code:0<6}"
        business_type = tr.select_one("td:nth-child(2)").text
        product = tr.select_one("td:nth-child(3)").text
        regi_date = tr.select_one("td:nth-child(4)").text
        settlement = tr.select_one("td:nth-child(5)").text
        ceo = tr.select_one("td:nth-child(6)").text
        homepage = tr.select_one("td:nth-child(7) > a")['href'] if tr.select_one("td:nth-child(7) > a") != None else "홈페이지 없음"
        region = tr.select_one("td:nth-child(8)").text
        values = (stock_type, company_name, code, business_type, product, regi_date, settlement, ceo, homepage, region)
        page_result.append(values)
    return page_result

In [8]:
result = {}
cols = ['주식종목', '회사명','종목코드','업종','주요제품','상장일','결산월',
       '대표자명','홈페이지','지역']
page = 1
total_page = 1

while page <= total_page:
    print(f"{page}/{total_page} 수집중           ", end="\r")
    url ="https://kind.krx.co.kr/corpgeneral/corpList.do"
    payload = dict(method="searchCorpList", pageIndex=page, currentPageSize=100, orderMode=3, orderStat="D", searchType=13, fiscalYearEnd="all", location="all")
    r = requests.post(url, data=payload)
#     print(r.url)
#     print(r.status_code)
    soup = bs(r.content, 'lxml')
    if total_page == 1:
        total_page = int(soup.select_one(".info.type-00 > em").text.replace(",", "")) // 100 + 1

    page_result = info_extraction(soup.select("tbody > tr"))


    for items in page_result:
        for key, value in zip(cols, items):
            result.setdefault(key, []).append(value)
            
    page += 1

df = pd.DataFrame(result)
df.to_csv("./data/상장기업정보.csv", index=False, encoding="utf-8-sig")


In [9]:
df

,주식종목,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,유가증권,명인제약,317450,의약품 제조업,"신경정신계열, 구강계열 완제의약품",2025-10-01,12월,이행명,http://www.myunginph.co.kr,경기도
1,코스닥,KB제33호스팩,0072Z0,금융 지원 서비스업,기업인수합병,2025-09-30,12월,박성원,홈페이지 없음,서울특별시
2,코스닥,삼성스팩11호,0071M0,금융 지원 서비스업,기업인수합병,2025-09-30,12월,문소연,홈페이지 없음,서울특별시
3,코스닥,에스투더블유,488280,소프트웨어 개발 및 공급업,"공공/정부기관용 사이버안보 빅데이터 플랫폼, 기업 및 기관용 사이버위협 인텔리전스 ...",2025-09-19,12월,서상덕,http://s2w.inc/,경기도
4,코스닥,오아,342870,가전제품 및 정보통신장비 소매업,계절/생활/주방/건강가전 및 헬스/건강식품,2025-09-09,12월,김상무,http://www.oa-world.com,서울특별시
...,...,...,...,...,...,...,...,...,...,...
2758,유가증권,유한양행,000100,의약품 제조업,"의약품(삐콤씨, 안티푸라민, 렉라자, 로수바미브, 코푸시럽 등), 생활용품(유한락스...",1962-11-01,12월,대표이..,http://www.yuhan.co.kr,서울특별시
2759,유가증권,CJ대한통운,000120,도로 화물 운송업,"Contract Logistics, 포워딩, 항만하역, 해운, 택배국제특송, SCM...",1956-07-02,12월,신영수..,http://www.cjlogistics.com,서울특별시
2760,유가증권,경방,000050,종합 소매업,"섬유류(면사,면혼방사,면직물,면혼방직물,화섬사,화섬직물) 제조,도매,수출입",1956-03-03,12월,"김준,..",http://www.kyungbang.co.kr,서울특별시
2761,유가증권,유수홀딩스,000700,회사 본부 및 경영 컨설팅 서비스업,지주사업,1956-03-03,12월,송영규,http://www.eusu-holdings.com,서울특별시


In [10]:
df.to_csv("./data/상장기업정보2.csv", index=False, encoding="utf-8-sig")

In [5]:
from io import StringIO

In [6]:
df_list = []
for page in range(1, 29):
    url ="https://kind.krx.co.kr/corpgeneral/corpList.do"
    payload = dict(method="searchCorpList", pageIndex=page, currentPageSize=100, orderMode=3, orderStat="D", searchType=13, fiscalYearEnd="all", location="all")
    r = requests.post(url, data=payload)
    temp = pd.read_html(StringIO(r.text))
    df_list.append(temp[0])
df = pd.concat(df_list)


In [198]:
df.columns = ['회사명', '업종','주요제품','상장일','결산월',
       '대표자명','홈페이지','지역']
df

,회사명,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,명인제약,의약품 제조업,"신경정신계열, 구강계열 완제의약품",2025-10-01,12월,이행명,홈페이지 보기,경기도
1,KB제33호스팩,금융 지원 서비스업,기업인수합병,2025-09-30,12월,박성원,NaN,서울특별시
2,삼성스팩11호,금융 지원 서비스업,기업인수합병,2025-09-30,12월,문소연,NaN,서울특별시
3,에스투더블유,소프트웨어 개발 및 공급업,"공공/정부기관용 사이버안보 빅데이터 플랫폼, 기업 및 기관용 사이버위협 인텔리전스 ...",2025-09-19,12월,서상덕,홈페이지 보기,경기도
4,오아,가전제품 및 정보통신장비 소매업,계절/생활/주방/건강가전 및 헬스/건강식품,2025-09-09,12월,김상무,홈페이지 보기,서울특별시
...,...,...,...,...,...,...,...,...
58,유한양행,의약품 제조업,"의약품(삐콤씨, 안티푸라민, 렉라자, 로수바미브, 코푸시럽 등), 생활용품(유한락스...",1962-11-01,12월,대표이..,홈페이지 보기,서울특별시
59,CJ대한통운,도로 화물 운송업,"Contract Logistics, 포워딩, 항만하역, 해운, 택배국제특송, SCM...",1956-07-02,12월,신영수..,홈페이지 보기,서울특별시
60,경방,종합 소매업,"섬유류(면사,면혼방사,면직물,면혼방직물,화섬사,화섬직물) 제조,도매,수출입",1956-03-03,12월,"김준,..",홈페이지 보기,서울특별시
61,유수홀딩스,회사 본부 및 경영 컨설팅 서비스업,지주사업,1956-03-03,12월,송영규,홈페이지 보기,서울특별시


In [7]:
df.to_csv("./data/상장기업정보.csv", index=False, encoding="utf-8-sig")